# はじめに
+ [e-Stat](https://www.e-stat.go.jp)のIDを入れ、「ランタイム」＞「すべてのセルを実行」により、実行できるようになっています。
  + <a href="https://colab.research.google.com/drive/1aCvMSDlnJ1k29duweuiaiFOXe_-WQPt3">こちらからGoogleColab上で試すことが可能</a>です（実行にはGoogleアカウントが必要）。
+ 1.main_program.ipynbで、全国の人口データを使うためにご利用ください。
  + このプログラムはcsvを吐き出すだけですが、main_program.ipynbと組み合わせれば、グラフ出力もできます。ただし、教材とするにはestatのidが必要です。

# eStatを簡単に読み込めるようにしたプラグインをinstall

In [ ]:
pip install pandas-estat

In [ ]:
# 自分のidを入れて、使えるようにする。
import pandas as pd
from pandas_estat import set_appid
from pandas_estat import read_statsdata
set_appid("自分のid")##　自分のid　をe-Statの自分のアカウントのappidに置き換える。

最近10年分の10/1現在の人口を読み込む（eStatが更新によって変更が必要の可能性があります）

In [ ]:
populations = {
  2020: read_statsdata("0003448228"),##2020に、2020-がある
  2015: read_statsdata("0003459018"),##2015に、2015-2019まである
  2014: read_statsdata("0003104181"),##2014までは、各年のデータ
  2013: read_statsdata("0003094384"),
  2012: read_statsdata("0003080162")
}

In [ ]:
# 2012/10のを表示してみる
populations[2012]

In [ ]:
# 2012/10のうち18歳を表示してみる
popu=populations[2012]
popu[(popu.全国=="全国")&
     (popu.年齢各歳=="18歳")&
     (popu.人口=="総人口")]

In [ ]:
populist, over100, sumlist=[],[],[]
for year0 in populations:
    popu=populations[year0]
    suwr=set(popu["時間軸（年月日現在）"])
    suwr0 = [asd[:4] for asd in suwr]
    for year in suwr0:
        print(year)
#        display(popu)
        for sei in ["男", "女", "男女計"]:
            for i in range(100):
                li = popu[(popu.全国=="全国")&
                          (popu.年齢各歳==str(i)+"歳")&
                          (popu.人口=="総人口")&
                          (popu["時間軸（年月日現在）"]==str(year)+"年10月1日現在")&
                          (popu["男女別・性比"]==sei)].index
                populist.append(
                    {"year": year, "age": i, "sex": sei, "value": popu.at[list(li)[0], "value"]})
                ##100歳以上も保存
                li = popu[(popu.全国=="全国")&
                        (popu.年齢各歳=="100歳以上")&
                        (popu.人口=="総人口")&
                        (popu["時間軸（年月日現在）"]==str(year)+"年10月1日現在")&
                        (popu["男女別・性比"]==sei)].index
                over100.append(
                  {"year": year, "sex": sei, "value": popu.at[list(li)[0], "value"]})
                ##100歳以上も保存
                li = popu[(popu.全国=="全国")&
                        (popu.年齢各歳=="総数")&
                        (popu.人口=="総人口")&
                        (popu["時間軸（年月日現在）"]==str(year)+"年10月1日現在")&
                        (popu["男女別・性比"]==sei)].index
                sumlist.append(
                  {"year": year, "sex": sei, "value": popu.at[list(li)[0], "value"]})

## データの書き出し

In [ ]:
import pandas as pd
data = pd.DataFrame(populist).astype({"year":int, "age":int, "value":int})
data.to_csv("japan_pop.csv", index=False)
##GoogleColabの場合は、左メニューの「content」フォルダ内に「japan_pop.csv」が作られていると思います。
##もし、見つからない場合は「%pwd」をセルで実行し、現在のフォルダがどこかを確認し、探してください。